# DATA PROCESSING AND CALCULATION OF xG-ADJUSTED FPL POINTS

## Setup and pre-processing

In [1]:
# give the number of the latest FPL round
latest_gameweek = 20

In [2]:
# import basic libraries
import pandas as pd
import numpy as np
import json
import requests
from scipy.stats import poisson

# allow more data columns to be shown than usually
pd.set_option('max_columns',100)

In [3]:
# import player data 
filepath = '../data/fbref/player_stats_week' + str(latest_gameweek) + '.csv'
playerStats = pd.read_csv(filepath, index_col=0, skiprows=1)#, encoding='latin-1')
playerStats.head()

,Player,Nation,Pos,Squad,Age,Born,MP,Starts,Min,Gls,Ast,PK,PKatt,CrdY,CrdR,Gls.1,Ast.1,G+A,G-PK,G+A-PK,xG,npxG,xA,xG.1,xA.1,xG+xA,npxG.1,npxG+xA,Matches
Rk,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,Patrick van Aanholt\Patrick-van-Aanholt,nl NED,DF,Crystal Palace,28,1990,17,17,1444,2,0,1,1,0,0,0.12,0.00,0.12,0.06,0.06,1.5,0.8,0.9,0.09,0.06,0.15,0.05,0.10,Matches
2,Max Aarons\Max-Aarons,eng ENG,MF,Norwich City,19,2000,18,18,1620,0,0,0,0,4,0,0.00,0.00,0.00,0.00,0.00,0.3,0.3,2.9,0.02,0.16,0.18,0.02,0.18,Matches
3,Tammy Abraham\Tammy-Abraham,eng ENG,FW,Chelsea,21,1997,19,18,1493,12,3,0,0,2,0,0.72,0.18,0.90,0.72,0.90,9.0,9.0,2.1,0.54,0.12,0.67,0.54,0.67,Matches
4,Che Adams\Che-Adams,eng ENG,FW,Southampton,23,1996,15,8,634,0,0,0,0,0,0,0.00,0.00,0.00,0.00,0.00,1.5,1.5,0.3,0.21,0.04,0.25,0.21,0.25,Matches
5,Adrián\Adrian,es ESP,GK,Liverpool,32,1987,10,8,783,0,0,0,0,1,0,0.00,0.00,0.00,0.00,0.00,0.0,0.0,0.0,0.00,0.00,0.00,0.00,0.00,Matches


In [4]:
# import team data and pre-process
filepath = '../data/fbref/team_stats_week' + str(latest_gameweek) + '.csv'
teamStats = pd.read_csv(filepath, index_col=0, encoding='latin-1')

# change team names to match convention used in the FPL data
teamStats.loc[teamStats['Squad']=='Brighton & Hove Albion','Squad'] = 'Brighton'
teamStats.loc[teamStats['Squad']=='Manchester United','Squad'] = 'Manchester Utd'
teamStats.loc[teamStats['Squad']=='Newcastle United','Squad'] = 'Newcastle Utd'
teamStats.loc[teamStats['Squad']=='Sheffield United','Squad'] = 'Sheffield Utd'
teamStats.loc[teamStats['Squad']=='West Ham United','Squad'] = 'West Ham'
teamStats.loc[teamStats['Squad']=='Tottenham Hotspur','Squad'] = 'Tottenham'
teamStats.loc[teamStats['Squad']=='Wolverhampton Wanderers','Squad'] = 'Wolves'

teamStats.head()

,Squad,MP,W,D,L,GF,GA,GDiff,Pts,xG,xGA,xGDiff,xGDiff/90,Last 5,Attendance,Top Team Scorer,Goalkeeper,Notes
Rk,,,,,,,,,,,,,,,,,,
1,Liverpool,19,18,1,0,47,14,33,55,35.7,17.5,18.2,0.96,W W W W W,53098,Sadio ManÃ© - 10,Alisson,NaN
2,Leicester City,20,13,3,4,43,19,24,42,32.5,24.2,8.3,0.41,W D L L W,32046,Jamie Vardy - 17,Kasper Schmeichel,NaN
3,Manchester City,20,13,2,5,54,23,31,41,48.2,20.1,28.1,1.41,L W W L W,54384,Raheem Sterling - 11,Ederson,NaN
4,Chelsea,20,11,2,7,35,28,7,35,33.6,20.3,13.3,0.66,L L W L W,40567,Tammy Abraham - 12,Kepa Arrizabalaga,NaN
5,Manchester Utd,20,8,7,5,32,23,9,31,31.6,18.9,12.7,0.63,W D L W W,72442,Marcus Rashford - 12,David de Gea,NaN


In [5]:
# fetch FPL data online
data = json.loads(requests.get('https://fantasy.premierleague.com/api/bootstrap-static/').text)
df = pd.DataFrame(data['elements'])
df.set_index('id',inplace=True)

# fetch data locally
#df = pd.read_csv('../data/data_week' + str(latest_gameweek) + '.csv', index_col=0)#,encoding='latin-1')

df.head()

,assists,bonus,bps,chance_of_playing_next_round,chance_of_playing_this_round,clean_sheets,code,cost_change_event,cost_change_event_fall,cost_change_start,cost_change_start_fall,creativity,dreamteam_count,element_type,ep_next,ep_this,event_points,first_name,form,goals_conceded,goals_scored,ict_index,in_dreamteam,influence,minutes,news,news_added,now_cost,own_goals,penalties_missed,penalties_saved,photo,points_per_game,red_cards,saves,second_name,selected_by_percent,special,squad_number,status,team,team_code,threat,total_points,transfers_in,transfers_in_event,transfers_out,transfers_out_event,value_form,value_season,web_name,yellow_cards
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,1,0,35,100.0,100.0,0,69140,0,0,-3,3,1.2,0,2,0.9,0.9,1,Shkodran,0.9,4,0,8.8,False,32.2,170,,2019-11-28T23:00:21.541666Z,52,0,0,0,69140.jpg,2.0,0,0,Mustafi,0.3,False,None,a,1,3,54.0,6,8171,95,32845,141,0.2,1.2,Mustafi,0
2,0,0,26,75.0,75.0,0,98745,0,0,-1,1,22.7,0,2,0.0,0.0,0,Héctor,0.0,6,0,6.9,False,30.8,262,Hamstring injury - 75% chance of playing,2019-12-09T20:00:21.228098Z,54,0,0,0,98745.jpg,0.3,0,0,Bellerín,0.2,False,None,d,1,3,15.0,1,34899,63,36164,428,0.0,0.2,Bellerín,2
3,2,1,196,0.0,0.0,1,111457,0,0,-3,3,149.6,0,2,0.0,0.0,0,Sead,1.1,17,0,37.8,False,174.4,928,Ankle injury - Expected back 11 Jan,2019-12-15T19:30:19.136195Z,52,0,0,0,111457.jpg,1.9,0,0,Kolasinac,0.5,False,None,i,1,3,55.0,27,46981,9,115767,764,0.2,5.2,Kolasinac,2
4,2,3,171,100.0,100.0,2,154043,0,0,-4,4,148.0,1,2,1.4,1.4,0,Ainsley,1.4,17,0,38.9,False,211.2,940,,2019-09-22T18:00:10.824841Z,46,0,0,0,154043.jpg,2.5,1,0,Maitland-Niles,2.5,False,None,a,1,3,35.0,27,553725,1843,576480,2783,0.3,5.9,Maitland-Niles,2
5,0,2,240,0.0,75.0,2,39476,0,0,-1,1,31.4,0,2,0.0,0.4,0,Sokratis,0.6,24,1,46.3,False,359.2,1426,Head injury - Expected back 11 Jan,2019-12-26T17:00:15.332949Z,49,0,0,0,39476.jpg,2.2,0,0,Papastathopoulos,1.3,False,None,i,1,3,71.0,35,108153,23,154952,2491,0.1,7.1,Sokratis,5


## Probability to keep a clean sheet

Here, we estimate for each team the probability that the team keeps a clean sheet (against average opposition). We do this by first calculating the expected goals allowed per game for each team. Then, we assume that conceding goals follows a Poisson distribution, from which we then get the desired probability.

In [6]:
teamStats['xGA per game'] = teamStats['xGA'] / teamStats['MP']
teamStats['probability no goals allowed'] = poisson.pmf(0,teamStats['xGA per game'])
teamStats

,Squad,MP,W,D,L,GF,GA,GDiff,Pts,xG,xGA,xGDiff,xGDiff/90,Last 5,Attendance,Top Team Scorer,Goalkeeper,Notes,xGA per game,probability no goals allowed
Rk,,,,,,,,,,,,,,,,,,,,
1,Liverpool,19,18,1,0,47,14,33,55,35.7,17.5,18.2,0.96,W W W W W,53098,Sadio ManÃ© - 10,Alisson,NaN,0.921053,0.398100
2,Leicester City,20,13,3,4,43,19,24,42,32.5,24.2,8.3,0.41,W D L L W,32046,Jamie Vardy - 17,Kasper Schmeichel,NaN,1.210000,0.298197
3,Manchester City,20,13,2,5,54,23,31,41,48.2,20.1,28.1,1.41,L W W L W,54384,Raheem Sterling - 11,Ederson,NaN,1.005000,0.366045
4,Chelsea,20,11,2,7,35,28,7,35,33.6,20.3,13.3,0.66,L L W L W,40567,Tammy Abraham - 12,Kepa Arrizabalaga,NaN,1.015000,0.362402
5,Manchester Utd,20,8,7,5,32,23,9,31,31.6,18.9,12.7,0.63,W D L W W,72442,Marcus Rashford - 12,David de Gea,NaN,0.945000,0.388680
6,Tottenham,20,8,6,6,36,29,7,30,25.6,24.2,1.4,0.07,W W L W D,59308,Harry Kane - 11,Paulo Gazzaniga,NaN,1.210000,0.298197
7,Wolves,20,7,9,4,29,25,4,30,25.5,20.8,4.7,0.23,D L W W L,31287,RaÃºl JimÃ©nez - 8,Rui PatrÃ­cio,NaN,1.040000,0.353455
8,Sheffield Utd,20,7,8,5,23,19,4,29,24.2,21.0,3.2,0.16,W W W D L,30799,Lys Mousset - 5,Dean Henderson,NaN,1.050000,0.349938
9,Crystal Palace,20,7,6,7,18,22,-4,27,18.6,26.4,-7.7,-0.39,D D L W D,25058,Jordan Ayew - 5,Vicente Guaita,NaN,1.320000,0.267135


## xG-adjusted points

Next, we determine for each player their 'adjusted points'. To do this, we first subtract for each player all the points they have accumulated through goals, assists and clean sheets. Then, we add points for each player based on their expected goals, assists and clean sheets. This gives a much improved estimate of each player's true point generating capability. 

In [7]:
def incorporate_xG(indicator, ix):
    xG = playerStats.loc[indicator, 'xG'].values[0]
    if df.loc[ix, 'element_type']<=2:
        df.loc[ix, 'adjusted points'] =  df.loc[ix, 'total_points'] -  6 * (df.loc[ix, 'goals_scored'] - xG)
    elif df.loc[ix, 'element_type']==3:
        df.loc[ix, 'adjusted points'] =  df.loc[ix, 'total_points'] -  5 * (df.loc[ix, 'goals_scored'] - xG)
    elif df.loc[ix, 'element_type']==4:
        df.loc[ix, 'adjusted points'] =  df.loc[ix, 'total_points'] -  4 * (df.loc[ix, 'goals_scored'] - xG)

In [8]:
# always run 'team_xGA' AFTER 'incorporate_xG'
def team_xGA(indicator, ix):
    team = team_names[df.loc[ix, 'team']-1]
    clean_sheets = df.loc[ix, 'clean_sheets']
    probability_cleanSheet = teamStats.loc[teamStats['Squad']==team, 'probability no goals allowed'].values[0]
    if df.loc[ix, 'element_type']<=2:
        df.loc[ix, 'adjusted points'] =  df.loc[ix, 'adjusted points'] -  \
                    4 * (df.loc[ix, 'clean_sheets'] - df.loc[ix, 'games played']*probability_cleanSheet)
    elif df.loc[ix, 'element_type']==3:
        df.loc[ix, 'adjusted points'] =  df.loc[ix, 'adjusted points'] -  \
                    (df.loc[ix, 'clean_sheets'] - df.loc[ix, 'games played']*probability_cleanSheet)

A player who gives an assist that directly leads to a shot, is assigned the xG-value of the shot in xA (expected assists), i.e. xA is a measure of 'goal assists'. In FPL, however, the definition of an assist is somewhat more relaxed, e.g. goals resulting from a rebound of parried shot will award an assist to the player making the initial shot. For this reason, we calculate the total number of assists awarded in FPL and the total sum of xA of all players and get an estimate of the proportion of assists that xA covers in FPL. Then, we modify that proportion of players assists based on their xA.

In [9]:
# always run 'xA' AFTER 'incorporate_xG'
def xA(indicator, ix):
    xA = playerStats.loc[indicator, 'xA'].values[0]
    df.loc[ix, 'adjusted points'] =  df.loc[ix, 'adjusted points'] -  3 * (xA_proportion*df.loc[ix, 'assists'] - xA)

In [10]:
team_names = np.sort(playerStats['Squad'].unique())
xA_proportion = playerStats['xA'].sum()/df['assists'].sum()
df['points_per_game'] = df['points_per_game'].astype(float)
df['games played'] = df['total_points']/df['points_per_game']
#df['games played'] = df['minutes']/90.0
xA_proportion

0.7539419087136934

## Main loop for assigning adjusted points

Below is the main loop where we calculate adjusted points for each player. Calculation of the adjusted points itself is straightforward, but there is some work required to match players in two different data sets. Comparing player names in both data sets gives unique matches in many cases, but some special cases need to be covered through individual solutions.

In [11]:
for ix in df[df['minutes']>0].index:    
    name = df.loc[ix, 'web_name'].lower().replace(' ', '').replace('-', '').replace('ü', 'u').replace('ö', 'o').\
                        replace('ä', 'a')
    indicator = playerStats['Player'].str.lower().str.replace(' ', '').str.replace('-', '').str.contains(name)
    if playerStats.loc[indicator].shape[0]==1:
        incorporate_xG(indicator, ix)
        team_xGA(indicator, ix)
        xA(indicator, ix)
    elif playerStats.loc[indicator].shape[0]==0:
        first_name = df.loc[ix, 'first_name'].lower().replace(' ', '').replace('-', '').replace('ü', 'u')\
                                                                    .replace('ö', 'o').replace('ä', 'a')
        first_name_indicator = playerStats['Player'].str.lower().str.replace(' ', '').str.replace('-', '')\
                                                                            .str.contains(first_name)
        if playerStats.loc[first_name_indicator].shape[0]==1:
            incorporate_xG(first_name_indicator, ix)
            team_xGA(first_name_indicator, ix)
            xA(first_name_indicator, ix)
        else:
            names = ['rodrigo','garcia','chicharito']
            names_playerStats_index = [76, 380, 335]
            if name in names:
                name_ix = names.index(name)
                exceptional_case_indicator = playerStats.index == names_playerStats_index[name_ix]
                incorporate_xG(exceptional_case_indicator, ix)
                team_xGA(exceptional_case_indicator, ix)
                xA(exceptional_case_indicator, ix)
            else:
                print(str(ix) + ': no player found.')
    elif playerStats.loc[indicator].shape[0]>1:
        full_name = df.loc[ix, 'first_name'].lower().replace(' ', '').replace('-', '') \
            + df.loc[ix, 'second_name'].lower().replace(' ', '').replace('-', '')
        full_name_indicator = playerStats['Player'].str.lower().str.replace(' ', '').str.replace('-', '')\
                                                                            .str.contains(full_name)
        if playerStats.loc[full_name_indicator].shape[0]==1:
            incorporate_xG(full_name_indicator, ix)
            team_xGA(full_name_indicator, ix)
            xA(full_name_indicator, ix)
        else:
            team = team_names[df.loc[ix, 'team']-1]
            team_indicator = playerStats['Squad']==team
            if playerStats.loc[indicator & team_indicator].shape[0]==1:
                incorporate_xG(indicator & team_indicator, ix)
                team_xGA(indicator & team_indicator, ix)
                xA(indicator & team_indicator, ix)
            else:
                if name=='son':
                    exceptional_case_indicator = playerStats.index == 176
                    incorporate_xG(exceptional_case_indicator, ix)
                    team_xGA(exceptional_case_indicator, ix)
                    xA(exceptional_case_indicator, ix)
                else:
                    print(str(ix) + ': non-unique name.')
    else:
        print(str(ix) + 'Player not found')

In [12]:
df['adjusted points per game'] = df['adjusted points'] / df['games played']

In [13]:
# give a sorted list showing the players with highest 'adjusted points per game'
df[['web_name', 'games played','total_points', 'points_per_game','adjusted points','adjusted points per game']]\
                                .sort_values(by='adjusted points per game', ascending=False)

,web_name,games played,total_points,points_per_game,adjusted points,adjusted points per game
id,,,,,,
386,Chicharito,2.000000,9,4.5,23.200000,11.600000
191,Salah,15.942029,110,6.9,105.599215,6.623951
192,Mané,18.082192,132,7.3,117.803911,6.514913
215,De Bruyne,19.041096,139,7.3,122.404331,6.428429
281,McGovern,2.000000,11,5.5,12.623268,6.311634
166,Vardy,18.947368,144,7.6,117.190871,6.185074
214,Sterling,19.122807,109,5.7,112.176149,5.866092
233,Rashford,20.000000,120,6.0,117.190871,5.859544
182,Alexander-Arnold,18.928571,106,5.6,109.723582,5.796718


In [14]:
# save data
filepath = '../data/data_week' + str(latest_gameweek) + str('.csv')
df.to_csv(filepath)

filepath = '../data/fbref/team_stats_week' + str(latest_gameweek) + '.csv'
teamStats.to_csv(filepath)

Below we check how well the total xG matches the total scored goals.

In [15]:
playerStats['xG'].sum()

535.3000000000003

In [16]:
df['goals_scored'].sum()

541

In [17]:
playerStats['xG'].sum()/df['goals_scored'].sum()

0.9894639556377085